# Analysis

In [115]:
from sklearn.model_selection import train_test_split
from analytics_utils.interpolate import interpolate
from sklearn.metrics import classification_report
from sklearn import preprocessing
import tensorflow as tf
import matplotlib
import joblib

from analytics_utils.describe_data import describe_data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

## Global Variables

In [3]:
ARGS = {
    "merra2_path": "dataset/extract/MERRA2/",
    "rmd_path": "dataset/extract/Reference_Monitor_Data/LosAngeles.csv",
    "aqs_path": "dataset/extract/AirQualitySystem.csv",
}

## Dataframes

__MERRA2__:

- Within each file are 24 hourly measurements for each of the 22 station locations
- Fields
  - Station – Name of ground monitor for data row
  - Lat – Latitude (degrees north) of station
  - Lon – Longitude (degrees east) of station
  - SRadius – Search radius (km) for nearest MERRA grid point to station
  - MERRALat – Latitude (degrees north) of nearest MERRA grid point to station
  - MERRAlon – Longitude (degrees east) of nearest MERRA grid point to station
  - IDXi – I index of MERRA grid point
  - IDXj – J index of MERRA grid point
  - PS – Surface pressure (Pa)
  - QV10m – Specific humidity at 10 m above surface (kg/kg)   	(multiplied by 1000.0)
  - Q500 - Specific humidity at 500 mbar pressure (kg/kg) 		(multiplied by 1000.0)
  - Q850 – Specific humidity at 850 mbar pressure (kg/kg) 		(multiplied by 1000.0)
  - T10m – Temperature at 10 m above surface (Kelvin)
  - T500 – Temperature at 500 mbar pressure (Kelvin)
  - T850 – Temperature at 850 mbar pressure (Kelvin)
  - Wind – Surface wind speed (m/s)
  - BCSMASS – Black Carbon mass concentration at surface (μg/m3)
  - DUSMASS25 – Dust surface mass PM 2.5 concentration at surface (μg/m3)
  - OCSMASS – Organic carbon mass concentration at surface (μg/m3)
  - SO2SMASS – Sulphur dioxide mass concentration at surface (μg/m3)
  - SO4SMASS – Sulphate aerosol mass concentration at surface (μg/m3)
  - SSSMASS25 – Sea Salt surface mass concentration PM 2.5 (μg/m3)
  - TOTEXTTAU – Total aerosol extinction AOT @ 550 nm (unitless)
  - UTC_DATE – YearMonthDay (GMT date)
  - UTC_TIME – Time of sample (hours) (GMT time)

In [26]:
# Dataframe
columns = [
    "Station",
    "Lat",
    "Lon",
    "SRadius",
    "PS",
    "QV10m",
    "Q500",
    "Q850",
    "T10m",
    "T500",
    "T850",
    "WIND",
    "BCSMASS",
    "DUSMASS25",
    "OCSMASS",
    "SO2SMASS",
    "SO4SMASS",
    "SSSMASS25",
    "TOTEXTTAU",
    "UTC_DATE",
    "UTC_TIME"
]
df_merra2 = pd.concat(
    [pd.read_csv(
        ARGS["merra2_path"] + _,
        usecols=columns
    ) for _ in os.listdir(ARGS["merra2_path"])],
    ignore_index=True,
)

df_merra2 = df_merra2[
    ~df_merra2["Station"].isin([
        "USDiplomaticPost:AddisAbabaCentral",
        "USDiplomaticPost:AddisAbabaSchool",
        "AnandVihar",
        "DelhiTechnologicalUniversity",
        "IHBAS",
        "IncomeTaxOffice",
        "MandirMarg",
        "NSITDwarka",
        "PunjabiBagh",
        "RKPuram",
        "RKPuram",
        "Sector16AFaridabad",
        "Shadipur",
        "USDiplomaticPost:NewDelhi",
        "VikasSadanGurgaon-HSPCB"
    ])]
df_merra2["UTC_TIME"] = df_merra2["UTC_TIME"].apply(lambda x: f"{}")
# df_merra2[-100:].to_csv("temp.json")

,Station,Lat,Lon,SRadius,PS,QV10m,Q500,Q850,T10m,T500,...,WIND,BCSMASS,DUSMASS25,OCSMASS,SO2SMASS,SO4SMASS,SSSMASS25,TOTEXTTAU,UTC_DATE,UTC_TIME
48,710NearRoad,33.8597,-118.201,17.1,96817.875,5.94036,0.19509,2.51205,299.374,266.326,...,3.086,0.49181,1.00090,2.48474,13.48235,2.96132,1.70439,0.074,20180922,0.5
49,710NearRoad,33.8597,-118.201,17.1,96839.258,6.11589,0.19538,2.89756,297.238,266.407,...,2.181,0.61709,0.97634,2.89492,15.94890,3.08504,1.71485,0.072,20180922,1.5
50,710NearRoad,33.8597,-118.201,17.1,96868.656,6.69963,0.19652,3.34065,294.951,266.450,...,1.911,0.81695,0.93087,3.52611,18.85201,3.18150,1.76669,0.070,20180922,2.5
51,710NearRoad,33.8597,-118.201,17.1,96916.977,6.98969,0.19378,3.93978,294.396,266.469,...,1.333,0.99317,0.88608,4.05089,21.23124,3.19870,1.73395,0.069,20180922,3.5
52,710NearRoad,33.8597,-118.201,17.1,96948.766,7.04901,0.18722,4.53671,294.229,266.452,...,0.880,1.15642,0.85106,4.54020,23.26851,3.20055,1.67893,0.068,20180922,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201163,SouthLongBeach,33.7922,-118.175,23.6,96647.727,10.00740,0.47602,5.53672,295.289,264.900,...,4.603,0.35709,3.07864,2.13458,22.52636,3.55614,2.83035,0.164,20180605,19.5
201164,SouthLongBeach,33.7922,-118.175,23.6,96613.555,10.36982,0.56769,5.67992,294.911,265.017,...,4.776,0.39131,3.08137,2.19643,21.10028,4.98224,2.91493,0.186,20180605,20.5
201165,SouthLongBeach,33.7922,-118.175,23.6,96586.203,10.26685,0.68713,5.66997,294.457,265.112,...,4.623,0.38210,3.04863,2.08547,19.55777,5.27873,3.04044,0.181,20180605,21.5
201166,SouthLongBeach,33.7922,-118.175,23.6,96566.266,9.82056,0.66950,5.61176,293.911,265.077,...,4.297,0.40086,3.06954,2.07638,18.50276,5.93910,2.94585,0.172,20180605,22.5


__Reference_Monitor_Data__:

- contains historical measurements of ground pollutants at each of the 22 locations for various time periods between 2016 and 2019. Each file contains measurements of PM2.5, PM10, and trace gas pollutants for time periods and sampling intervals that vary by site. Not all sites have all data for the full period.

In [89]:
# Dataframe
columns = ["date", "parameter", "value", "coordinates"]
df_rmd = pd.read_csv(
    ARGS["rmd_path"],
    usecols=columns,
)

coordinates = df_rmd['coordinates']
lat = [float(x.split(",")[0][10:]) for x in coordinates]
lon = [float(x.split(",")[1][11:-1]) for x in coordinates]

datetime = df_rmd['date']
date = [x[5:15] for x in datetime]
ano = [x[:4] for x in date]
mes = [x[5:7] for x in date]
dia = [x[9:] for x in date]

time = [x[16:24] for x in datetime]
hora = [x[:2] for x in time]

gmt = [x[-7:-1] for x in datetime]

df_rmd['Lat'] = lat
df_rmd['Long'] = lon
df_rmd['date'] = date
df_rmd['day'] = dia
df_rmd['month'] = mes
df_rmd['year'] = ano
df_rmd['time'] = time
df_rmd['hour'] = hora
df_rmd['datetime'] = df_rmd[["date", "time"]].apply(lambda x: ' '.join(x), axis=1)
df_rmd['gmt'] = gmt
df_rmd = df_rmd.drop(["coordinates", "date", "time"], axis=1)
df_rmd = df_rmd.set_index("datetime")
df_rmd.head()

,parameter,value,Lat,Long,day,month,year,hour,gmt
datetime,,,,,,,,,
2017-08-11 00:00:00,co,0.340,34.136475,-117.923965,1,08,2017,00,-08:00
2017-08-11 00:00:00,no2,0.015,34.136475,-117.923965,1,08,2017,00,-08:00
2017-08-11 00:00:00,o3,0.061,34.136475,-117.923965,1,08,2017,00,-08:00
2017-08-11 00:00:00,co,0.240,34.143900,-117.850800,1,08,2017,00,-08:00
2017-08-11 00:00:00,no2,0.012,34.143900,-117.850800,1,08,2017,00,-08:00


In [90]:
df_rmd.shape

(986034, 9)

drop all negatives values

In [91]:
df_rmd_clear = df_rmd.drop(df_rmd[df_rmd["value"] < 0.0].index)
df_rmd_clear

,parameter,value,Lat,Long,day,month,year,hour,gmt
datetime,,,,,,,,,
2017-08-11 00:00:00,co,0.340,34.136475,-117.923965,1,08,2017,00,-08:00
2017-08-11 00:00:00,no2,0.015,34.136475,-117.923965,1,08,2017,00,-08:00
2017-08-11 00:00:00,o3,0.061,34.136475,-117.923965,1,08,2017,00,-08:00
2017-08-11 00:00:00,co,0.240,34.143900,-117.850800,1,08,2017,00,-08:00
2017-08-11 00:00:00,no2,0.012,34.143900,-117.850800,1,08,2017,00,-08:00
...,...,...,...,...,...,...,...,...,...
2019-04-03 20:00:00,co,0.110,33.629990,-117.675870,3,04,2019,20,-08:00
2019-04-03 20:00:00,o3,0.042,33.629990,-117.675870,3,04,2019,20,-08:00
2019-04-03 20:00:00,co,0.100,33.925060,-117.952580,3,04,2019,20,-08:00


Row to Columns

In [104]:
aux = df_rmd_clear[df_rmd_clear["parameter"] == "so2"]
aux["value"].unique()

array([0.   , 0.001, 0.002, 0.004, 0.003, 0.01 , 0.007, 0.005, 0.006,
       0.008, 0.009, 0.016, 0.014, 0.018, 0.022, 0.013])

Ungroup dataframe

In [94]:
parameters = ['co', 'no2', 'o3', 'pm10', 'pm25', 'so2']
dfs = [df_rmd_clear[df_rmd_clear["parameter"] == _] for _ in parameters]
for i in range(len(dfs)):
    dfs[i] = dfs[i].rename(columns={'value': dfs[i]["parameter"][0]})
    dfs[i] = dfs[i].drop("parameter", axis=1)
dfs

[                       co        Lat        Long day month  year hour     gmt
 datetime                                                                     
 2017-08-11 00:00:00  0.34  34.136475 -117.923965   1    08  2017   00  -08:00
 2017-08-11 00:00:00  0.24  34.143900 -117.850800   1    08  2017   00  -08:00
 2017-08-11 00:00:00  0.24  34.050600 -118.455300   1    08  2017   00  -08:00
 2017-08-11 00:00:00  0.29  34.066430 -118.226750   1    08  2017   00  -08:00
 2017-08-11 00:00:00  0.15  34.199200 -118.533100   1    08  2017   00  -08:00
 ...                   ...        ...         ...  ..   ...   ...  ...     ...
 2019-04-03 20:00:00  0.06  33.955070 -118.430460   3    04  2019   20  -08:00
 2019-04-03 20:00:00  0.17  34.383300 -118.528300   3    04  2019   20  -08:00
 2019-04-03 20:00:00  0.15  33.830585 -117.938510   3    04  2019   20  -08:00
 2019-04-03 20:00:00  0.11  33.629990 -117.675870   3    04  2019   20  -08:00
 2019-04-03 20:00:00  0.10  33.925060 -117.952580   

In [118]:
columns = ["datetime", "Lat", "Long", "day", "month", "year", "hour", "gmt"]
df = dfs[0]
for i in range(1, len(dfs)):
    df = df.merge(dfs[i], left_on=columns, right_on=columns, how='outer')
# df.to_json("temp.json")
# df.to_csv("outer.csv")
df

,co,Lat,Long,day,month,year,hour,gmt,no2,o3,pm10,pm25,so2
datetime,,,,,,,,,,,,,
2017-08-11 00:00:00,0.34,34.136475,-117.923965,1,08,2017,00,-08:00,0.015,0.061,NaN,NaN,NaN
2017-08-11 00:00:00,0.24,34.143900,-117.850800,1,08,2017,00,-08:00,0.012,0.071,58.0,11.9,NaN
2017-08-11 00:00:00,0.24,34.050600,-118.455300,1,08,2017,00,-08:00,0.002,0.043,NaN,NaN,NaN
2017-08-11 00:00:00,0.29,34.066430,-118.226750,1,08,2017,00,-08:00,0.010,0.051,33.0,17.0,0.000
2017-08-11 00:00:00,0.15,34.199200,-118.533100,1,08,2017,00,-08:00,0.004,0.069,NaN,19.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-13 19:00:00,NaN,34.066430,-118.226750,3,08,2019,19,-08:00,NaN,NaN,NaN,NaN,0.000
2019-08-13 21:00:00,NaN,34.066430,-118.226750,3,08,2019,21,-08:00,NaN,NaN,NaN,NaN,0.001
2019-08-24 08:00:00,NaN,34.066430,-118.226750,4,08,2019,08,-08:00,NaN,NaN,NaN,NaN,0.000


## NEURAL NETWORK

In [109]:
df.sort_values("datetime")

,co,Lat,Long,day,month,year,hour,gmt,no2,o3,pm10,pm25,so2
datetime,,,,,,,,,,,,,
2016-11-15 20:00:00,0.71,34.06690,-118.24170,5,11,2016,20,-08:00,0.060,0.027,78.0,45.0,0.001
2016-11-16 00:00:00,0.55,34.06690,-118.24170,6,11,2016,00,-08:00,0.037,0.029,59.0,27.0,0.001
2016-11-16 01:00:00,0.57,34.06690,-118.24170,6,11,2016,01,-08:00,0.040,0.019,57.0,22.0,0.001
2016-11-16 15:00:00,1.02,34.06690,-118.24170,6,11,2016,15,-08:00,0.034,0.001,78.0,32.0,0.001
2016-11-16 16:00:00,0.53,34.06690,-118.24170,6,11,2016,16,-08:00,0.027,0.007,67.0,37.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-06 11:00:00,0.23,34.06643,-118.22675,6,06,2019,11,-08:00,0.009,0.026,20.0,17.0,0.000
2019-06-06 12:00:00,0.23,34.06643,-118.22675,6,06,2019,12,-08:00,0.011,0.025,24.0,14.0,0.000
2019-06-06 13:00:00,0.23,34.06643,-118.22675,6,06,2019,13,-08:00,0.010,0.027,27.0,17.0,0.000
